In [1]:
#Initial environement of computations

import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests


number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [2]:
#Initial table set

#venus
Equation_center=HTable.read(189)
Equation_anomaly=HTable.read(190)
Anomaly_maxdist=HTable.read(250)
Anomaly_mindist=HTable.read(251)
Min_prop=HTable.read(252)
Target_equation=HTable.read(1363)



In [4]:
# Formating and initialising the Test tables

Hist_table=Target_equation.copy()
Test_recomp=Target_equation.copy()
Delta_recomp=Target_equation.copy()
for i in range(60):
    for j in range(36):
        Test_recomp[i][1][j][1]=Sexagesimal('0;')*u.degree
        Delta_recomp[i][1][j][1]=Sexagesimal('0;')*u.degree

In [5]:
for i in range(60):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=2):
        # Initial values for mean center 
        mean_center= Hist_table[i][0]*u.degree
                        
        # computation of the equation of center
        eq_center=Equation_center.get(mean_center)
            
    for j in range(36):
        with set_precision(tmode=TruncatureMode.TRUNC, pmode=2):

            # Initial values for mean argument
            mean_argument= Hist_table[0][1][j][0]*u.degree

            # Computation of the true argument
            true_argument=mean_argument-eq_center   

            # Computation of the true center
            true_center= mean_center+eq_center

            # computation of the first equation of anomaly
            eq_anomaly1=Equation_anomaly.get(true_argument)

            #correction of the equation of anomaly with the minuta proportionalia and diversity of diameter
            min_prop=Min_prop.get(true_center)
            if min_prop<=0:
                div_diam=Anomaly_maxdist.get(true_argument)
                prop_part_anomaly=div_diam*min_prop/(60*u.degree)
                eq_anomaly2=eq_anomaly1+prop_part_anomaly
            else:
                div_diam=Anomaly_mindist.get(true_argument)
                prop_part_anomaly=div_diam*min_prop/(60*u.degree)
                eq_anomaly2=eq_anomaly1+prop_part_anomaly

        # computation of the total equation
        with set_precision(tmode=TruncatureMode.ROUND, pmode=1):
            eq_total=eq_center+eq_anomaly2
            Test_recomp[i][1][j][1]=eq_total
            Delta_recomp[i][1][j][1]=60*(Hist_table.get(mean_center.value).get(mean_argument.value)-eq_total)/u.degree


In [6]:
# residuals visualisation and statistical summaries 1
A=Delta_recomp.to_pandas()
(A.mean().mean(), A.std().std())

(0.1787037037037037, 7.8371694774601455)

In [7]:
# residuals visualisation and statistical summaries 2
A.astype(float).style.background_gradient(axis=None)

,06 ;,12 ;,18 ;,24 ;,01s 00 ;,01s 06 ;,01s 12 ;,01s 18 ;,01s 24 ;,02s 00 ;,02s 06 ;,02s 12 ;,02s 18 ;,02s 24 ;,03s 00 ;,03s 06 ;,03s 12 ;,03s 18 ;,03s 24 ;,04s 00 ;,04s 06 ;,04s 12 ;,04s 18 ;,04s 24 ;,05s 00 ;,05s 06 ;,05s 12 ;,05s 18 ;,05s 24 ;,06s 00 ;,06s 06 ;,06s 12 ;,06s 18 ;,06s 24 ;,07s 00 ;,07s 06 ;,07s 12 ;,07s 18 ;,07s 24 ;,08s 00 ;,08s 06 ;,08s 12 ;,08s 18 ;,08s 24 ;,09s 00 ;,09s 06 ;,09s 12 ;,09s 18 ;,09s 24 ;,10s 00 ;,10s 06 ;,10s 12 ;,10s 18 ;,10s 24 ;,11s 00 ;,11s 06 ;,11s 12 ;,11s 18 ;,11s 24 ;,1r 00s 00 ;
00 ;,-2.000000,-2.000000,-3.000000,-2.000000,-2.000000,-3.000000,-3.000000,-2.000000,-2.000000,-2.000000,8.000000,-1.000000,0.000000,-1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,2.000000,3.000000,5.000000,2.000000,2.000000,2.000000,1.000000,0.000000,0.000000,0.000000,-1.000000,-2.000000,-2.000000,-2.000000,-5.000000,-4.000000,-3.000000,-2.000000,-3.000000,-2.000000,-2.000000,-2.000000,-3.000000,-2.000000,-2.000000,-2.000000,-1.000000,-2.000000,-1.000000,-1.000000,-2.000000,-1.000000,-1.000000,-2.000000,-1.000000,-1.000000,-2.000000,-2.000000,-2.000000
06 ;,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
12 ;,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
18 ;,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,-1.000000,0.000000,1.000000,0.000000,-1.000000,0.000000,1.000000,-1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,-1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000
24 ;,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,-1.000000,0.000000,1.000000,-1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,-1.000000,0.000000,0.000000,0.000000
01s 00 ;,0.000000,0.000000,0.000000,360.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-1.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,-1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
01s 06 ;,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,0.

In [8]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Venus_eq_TM.csv')
B.to_csv('Venus_eq_TM.zip', index=False, compression=compression_opts)